In [ ]:
!pip install web3
#!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 501 kB 13.7 MB/s 
     |████████████████████████████████| 82 kB 990 kB/s 
     |████████████████████████████████| 103 kB 84.9 MB/s 
     |████████████████████████████████| 1.1 MB 68.5 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 271 kB 72.5 MB/s 
     |████████████████████████████████| 144 kB 69.6 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 235 kB 61.8 MB/s 
     |████████████████████████████████| 2.3 MB 61.0 MB/s 
     |████████████████████████████████| 1.6 MB 90.9 MB/s 
     |████████████████████████████████| 1.9 MB 55.9 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=6958adf22e1a00b354ec340d491f387e1a977f48de6fd468ee4946fe

# Reading and Cleaning Data


In [ ]:
from numpy.ma.core import default_fill_value
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from web3 import Web3
import json
from datetime import datetime, timezone
import time
import pandas as pd

# Contract address and ABI details
address_contract = "0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA"
with open("pancake_pred_abi.json", "r") as myFile:
  data = myFile.read()
abi = json.loads(data)
#print(abi['result'])

# Provider
rpc_mainnet = "https://bsc-dataseed1.binance.org/"
w3 = Web3(Web3.HTTPProvider(rpc_mainnet))

account_address = "0x7F062ef11F2C17A0cc2F8615E01b319DaCC217Ad" 
pk = ""

# Contract
contract = w3.eth.contract(address=address_contract, abi=abi['result'])


# Current Epoch
current_epoch = contract.functions.currentEpoch().call()

# Init Variables
lookback = 30
starting_epoch = current_epoch - lookback
columns = ["epoch", "start_timestamp", "lock_timestamp", "close_timestamp", "lock_price", "close_price", "total_amount", "bull_amount", "bear_amount", "bull_ratio", "bear_ratio", "oracle_called"]
df = pd.DataFrame(columns=columns)
count = 0
for e in range(0, lookback):
  time.sleep(1)
  starting_epoch += 1
  count += 1

  # Get round object
  current_rounds_list = contract.functions.rounds(starting_epoch).call()

  #print(current_rounds_list)

  # Name items
  epoch = current_rounds_list[0]
  start_timestamp = current_rounds_list[1]
  lock_timestamp = current_rounds_list[2]
  close_timestamp = current_rounds_list[3]
  lock_price = current_rounds_list[4]
  close_price = current_rounds_list[5]
  lock_oracle_id = current_rounds_list[6]
  close_oracle_id = current_rounds_list[7]
  total_amount = current_rounds_list[8]
  bull_amount = current_rounds_list[9]
  bear_amount = current_rounds_list[10]
  oracle_called = current_rounds_list[-1]
  
  # Calculate Ratio
  total_amount_normal = round(float(Web3.fromWei(total_amount, "ether")), 5)
  bull_amount_normal = round(float(Web3.fromWei(bull_amount, "ether")), 5)
  bear_amount_normal = round(float(Web3.fromWei(bear_amount, "ether")), 5)

  # Payout Ratios
  if bull_amount_normal != 0 and bear_amount_normal != 0:
    bull_ratio = round(bull_amount_normal/bear_amount_normal, 2) + 1
    bear_ratio = round(bear_amount_normal/bull_amount_normal, 2) + 1

  row_dict = {
      "epoch": epoch,
      "start_timestamp": start_timestamp, 
      "lock_timestamp": lock_timestamp, 
      "close_timestamp": close_timestamp, 
      "lock_price": lock_price, 
      "close_price": close_price, 
      "total_amount": total_amount, 
      "bull_amount": bull_amount, 
      "bear_amount": bear_amount,
      "bull_ratio": bull_ratio,
      "bear_ratio": bear_ratio, 
      "oracle_called": oracle_called,
  }

  try:
    df = df.append([row_dict])
    df.to_csv("predictions.csv")
  except:
    print("Error saving")

In [ ]:
df = df.set_index([pd.Index([i for i in range(0,len(df))])])
df

,epoch,start_timestamp,lock_timestamp,close_timestamp,lock_price,close_price,total_amount,bull_amount,bear_amount,bull_ratio,bear_ratio,oracle_called
0,89500,1658612130,1658612430,1658612736,25525824071,25575900515,15566749602923859012,6708711632226251631,8858037970697607381,1.76,2.32,True
1,89501,1658612436,1658612736,1658613045,25575900515,25584905875,11644910510161378348,5423962579736714071,6220947930424664277,1.87,2.15,True
2,89502,1658612745,1658613045,1658613351,25584905875,25553000000,13768148141949962274,5167740550257069516,8600407591692892758,1.60,2.66,True
3,89503,1658613051,1658613351,1658613660,25553000000,25553584385,12056415234147356740,6851658092477836588,5204757141669520152,2.32,1.76,True
4,89504,1658613360,1658613660,1658613969,25553584385,25569037897,12582240949876438142,6244169726617078032,6338071223259360110,1.99,2.02,True
5,89505,1658613669,1658613969,1658614275,25569037897,25616000000,13090036082273320050,5889516524500719995,7200519557772600055,1.82,2.22,True
6,89506,1658613975,1658614275,1658614581,25616000000,25584269221,18537097857105051407,8653737297241102299,9883360559863949108,1.88,2.14,True
7,89507,1658614281,1658614581,1658614887,25584269221,25640000000,12369550761377692887,7173563971580025895,5195986789797666992,2.38,1.72,True
8,89508,1658614587,1658614887,1658615193,25640000000,25761013693,15165033789661740277,8941567316918794521,6223466472742945756,2.44,1.70,True
9,89509,1658614893,1658615193,1658615499,25761013693,25770487917,30167326795398955652,13846511667616634162,16320815127782321490,1.85,2.18,True


In [ ]:
up_or_down = []
odds_preds = []
matched = []
for index, row in df.iterrows():
  if df['lock_price'][index] > df['close_price'][index]:
    up_or_down.append(0)
  elif df['lock_price'][index] <= df['close_price'][index]:
    up_or_down.append(1)
  if df['bull_ratio'][index] >= 2:
    odds_preds.append(1)
  else:
    odds_preds.append(0)
  matched.append(up_or_down[index] == odds_preds[index])


In [ ]:
up_or_down

[0, 0, 1]

In [ ]:
df.insert(len(df.columns), "up_or_down", up_or_down)

In [ ]:
df.insert(len(df.columns), "odds_preds", odds_preds)

In [ ]:
df.insert(len(df.columns), 'matched', matched)

In [ ]:
df

,epoch,start_timestamp,lock_timestamp,close_timestamp,lock_price,close_price,total_amount,bull_amount,bear_amount,bull_ratio,bear_ratio,oracle_called,up_or_down,odds_preds,matched
0,89500,1658612130,1658612430,1658612736,25525824071,25575900515,15566749602923859012,6708711632226251631,8858037970697607381,1.76,2.32,True,1,0,False
1,89501,1658612436,1658612736,1658613045,25575900515,25584905875,11644910510161378348,5423962579736714071,6220947930424664277,1.87,2.15,True,1,0,False
2,89502,1658612745,1658613045,1658613351,25584905875,25553000000,13768148141949962274,5167740550257069516,8600407591692892758,1.60,2.66,True,0,0,True
3,89503,1658613051,1658613351,1658613660,25553000000,25553584385,12056415234147356740,6851658092477836588,5204757141669520152,2.32,1.76,True,1,1,True
4,89504,1658613360,1658613660,1658613969,25553584385,25569037897,12582240949876438142,6244169726617078032,6338071223259360110,1.99,2.02,True,1,0,False
5,89505,1658613669,1658613969,1658614275,25569037897,25616000000,13090036082273320050,5889516524500719995,7200519557772600055,1.82,2.22,True,1,0,False
6,89506,1658613975,1658614275,1658614581,25616000000,25584269221,18537097857105051407,8653737297241102299,9883360559863949108,1.88,2.14,True,0,0,True
7,89507,1658614281,1658614581,1658614887,25584269221,25640000000,12369550761377692887,7173563971580025895,5195986789797666992,2.38,1.72,True,1,1,True
8,89508,1658614587,1658614887,1658615193,25640000000,25761013693,15165033789661740277,8941567316918794521,6223466472742945756,2.44,1.70,True,1,1,True
9,89509,1658614893,1658615193,1658615499,25761013693,25770487917,30167326795398955652,13846511667616634162,16320815127782321490,1.85,2.18,True,1,0,False


In [ ]:
df.to_csv("predictions.csv")

# Send Tx


In [ ]:
def send_tx(side):
  # Variables
  chain_id = 56
  gas = 300000
  gas_price = Web3.toWei("5.5", "gwei")
  send_bnb = 0.01
  amount = Web3.toWei(send_bnb, "ether")

  # None
  nonce = w3.eth.getTransactionCount(account_address)

  # Build tx
  if side == 'bull':
    tx_build = contract.functions.betBull(current_epoch).buildTransaction({
        "chainId": chain_id,
        "value": amount,
        "gas": gas,
        "gasPrice": gas_price,
        "nonce": nonce,
    })

    if side == 'bear':
      tx_build = contract.functions.betBull(current_epoch).buildTransaction({
        "chainId": chain_id,
        "value": amount,
        "gas": gas,
        "gasPrice": gas_price,
        "nonce": nonce,
    })
      
    # Sign tx
    tx_signed = w3.eth.account.signTransaction(tx_build, private_key=pk)

    # Send transaction
    sent_tx = w3.eth.sendRawTransaction(tx_signed.sendRawTransaction)

    print(sent_tx)


send_tx("nonce")

In [ ]:
def look_to_trade():

    # Current Rounds information
    current_rounds_list = contract.functions.rounds(current_epoch).call()
    lock_timestamp = current_rounds_list[2]
    total_amount = current_rounds_list[8]
    bull_amount = current_rounds_list[9]
    bear_amount = current_rounds_list[10]

    # Get current timestamp
    dt = int(datetime.now(timezone.utc).timestamp())
    time_remaining = lock_timestamp - dt

    # Calculate Ratio
    total_amount_normal = round(float(Web3.fromWei(total_amount, "ether")), 5)
    bull_amount_normal = round(float(Web3.fromWei(bull_amount, "ether")), 5)
    bear_amount_normal = round(float(Web3.fromWei(bear_amount, "ether")), 5)

    # Ratios
    if bull_amount_normal != 0 and bear_amount_normal != 0:
        bull_ratio = round(bull_amount_normal / bear_amount_normal, 2) + 1
        bear_ratio = round(bear_amount_normal / bull_amount_normal, 2) + 1
    else:
        bull_ratio = 0
        bear_ratio = 0

    # Ratios
    print(bull_ratio, bear_ratio)

    # Place trade
    if time_remaining <= 10:
        if bull_ratio > bear_ratio:
            send_tx("bull")
        else:
            send_tx("bear")
        print("done")

In [ ]:
def claim_winnings(epoch):

    # Variables
    chain_id = 56
    gas = 300000
    gas_price = Web3.toWei("5.5", "gwei")

    # Nonce
    nonce = w3.eth.getTransactionCount(account_address)

    # Caim Winnings
    tx_build = contract.functions.claim([epoch]).buildTransaction({
        "chainId": chain_id,
        "gas": gas,
        "gasPrice": gas_price,
        "nonce": nonce
    })
    print(tx_build)

    # Sign transaction
    tx_signed = w3.eth.account.signTransaction(tx_build, private_key=pk)
    print(tx_signed)

    # Send transaction
    sent_tx = w3.eth.sendRawTransaction(tx_signed.rawTransaction)
    print(sent_tx)

claim_winnings(None)
# look_to_trade()

ValidationError: ignored